# Supplementary - Effect of coarse-graining and individual eigenmode-gradient correlations

Conducts analyses related to Supplementary Figure S8.

In [ ]:
import sys
sys.path.append("..")

from main import *
from zebrafish import *

from scipy.stats import zscore, spearmanr
from scipy.stats import percentileofscore
from brainsmash.mapgen.base import Base

# Setting paths

In [ ]:
atlas_path = '/home/anleg84/Documents/Atlas/Mapzebrain/'

atlas = Mapzebrain(atlas_path)
mask_tectum = atlas.get_region_mask(22)
mask_tectum[:, :, :284] = 0

In [ ]:
top_directory = '/media/anleg84/KINGSTON/Datasets/Geometry/Reprocess/'
try:
    datasets = identify_folders(top_directory, ['920nm'])
except:
    datasets = None

# Part 1: Varying the number of tectal nodes

Studying the effect of coarse-graining on geometric gradients.

In [ ]:
N_nodes = [200, 400, 600, 800, 1000]

for N in N_nodes:

    print(f'Computing geometric mapping for {N} tectal nodes...')

    nodes_tectum = np.load(f'../Files/nodes_tectum_right_{N}.npy') * 2  # Pre-computed in Figure6-Meshes.ipynb

    # Mapping neurons in tectal nodes
    N_in_tectum = []
    centroids_tectum = []
    for folder in datasets:
        data = load_data(folder)
        centroids = data['centroids_atlas']
        in_tectum = mask_tectum[centroids[:, 2], centroids[:, 1], centroids[:, 0]] > 0
        N_in_tectum.append(np.sum(in_tectum))
        centroids_tectum.append(centroids[in_tectum])

    # Verifying which nodes contain a sufficient amount of neurons across animals
    vectors = []
    for c in centroids_tectum:
        nn, _ = find_nearest_neighbors(centroids_tectum[0], nodes_tectum)
        vector = np.zeros((nodes_tectum.shape[0], ))
        vector[np.unique(nn)] = 1
        vectors.append(vector)
    vectors = np.stack(vectors, axis=0)
    sampled = (np.sum(vectors > 0, axis=0) >= 0.5 * len(datasets))
    nodes_tectum = nodes_tectum[sampled]

    # Computing tectal correlation matrices
    mask_tectum = atlas.get_region_mask(22)
    mask_tectum[:, :, :284] = 0 # Focusing on right hemisphere (x=284 is the saggital midplane)
    FC_matrices = []
    for i, folder in enumerate(datasets):
        data = load_data(folder)
        centroids = data['centroids_atlas']
        in_tectum = mask_tectum[centroids[:, 2], centroids[:, 1], centroids[:, 0]] > 0
        centroids_tectum = centroids[in_tectum]
        dff_tectum = data['timeseries'][in_tectum]
        dff_tectum = compute_dff_using_minfilter(dff_tectum, window=120, sigma1=3, sigma2=60)
        dff_tectum = filter_timeseries(dff_tectum, 2)
        nn, _ = find_nearest_neighbors(centroids_tectum, nodes_tectum)
        node_timeseries = np.zeros((nodes_tectum.shape[0], dff_tectum.shape[1]))
        for i in range(nodes_tectum.shape[0]):
            neurons_in_node = (nn == i) # Neurons whose nearest neighbor corresponds to the tectum node i
            if np.any(neurons_in_node):
                if np.sum(neurons_in_node) == 1:
                    node_timeseries[i] = dff_tectum[neurons_in_node]
                else:
                    node_timeseries[i] = np.mean(dff_tectum[neurons_in_node], axis=0)                         
        FC = np.corrcoef(node_timeseries)
        FC_matrices.append(FC)
    FC_matrices = np.stack(FC_matrices, axis=0)

    # Computing average correlation matrix
    FC = np.nanmean(FC_matrices[:], axis=0)
    FC[np.diag_indices(FC.shape[0])] = 0
    order = np.argsort(nodes_tectum[:, 1])

    # Plotting correlation matrix
    plt.figure(figsize=(5, 5), dpi=150)
    plt.imshow(FC[order, :][:, order], vmin=-0.25, vmax=0.25, cmap='coolwarm')
    plt.xlabel('Node $j$')
    plt.ylabel('Node $i$')
    plt.show()

    # Computing gradients and comparing with geometric eigenmodes
    N_modes = FC.shape[0]
    gradients, _ = diffusion_mapping(np.abs(FC), n_components=N_modes)
    vertices = np.load('../Files/tectum_vertices_right.npy') * 40
    vertices = np.stack([vertices[:, 2], vertices[:, 1], vertices[:, 0]], axis=1)
    eigenmodes = np.load('../Files/tectum_eigenmodes_right.npy')[1:]
    d = compute_distances(vertices, nodes_tectum)
    eigenmodes = eigenmodes[:, np.argmin(d, axis=0)]
    mode_similarity, mapping = compute_mode_similarity_matrix(eigenmodes[:30], gradients.T[:30], return_mapping=True)

    plt.figure(figsize=(5, 5), dpi=150)
    plt.imshow(np.abs(mode_similarity), cmap='Reds', vmin=0.1, vmax=1)
    plt.ylabel('Geometric modes')
    plt.xlabel('Functional gradients')
    plt.show()

    np.save(f'../Results/supp_similarity_matrix_{N}nodes.npy', mode_similarity)

Loading results from the previous cell

In [ ]:
matrices = [np.load('../Results/supp_similarity_matrix_200nodes.npy'),
            np.load('../Results/supp_similarity_matrix_400nodes.npy'),
            np.load('../Results/supp_similarity_matrix_600nodes.npy'),
            np.load('../Results/supp_similarity_matrix_800nodes.npy'),
            np.load('../Results/supp_similarity_matrix_1000nodes.npy')]

matrices = np.stack(matrices)

In [ ]:
%matplotlib inline

Plotting mode correlation matrices

In [ ]:
fig, ax = plt.subplots(1, 5, figsize=(15, 3))
ax[0].imshow(np.abs(matrices[0]), vmin=0, vmax=1, cmap='Reds')
ax[1].imshow(np.abs(matrices[1]), vmin=0.1, vmax=1, cmap='Reds')
ax[2].imshow(np.abs(matrices[2]), vmin=0.1, vmax=1, cmap='Reds')
ax[3].imshow(np.abs(matrices[3]), vmin=0.1, vmax=1, cmap='Reds')
ax[4].imshow(np.abs(matrices[4]), vmin=0.1, vmax=1, cmap='Reds')

# Part 2: Single-fish gradients

In [ ]:
nodes_tectum = np.load('../Files/nodes_tectum_right_sampled.npy')

#### Computing single-cell and nodal correlations

In [ ]:
FC_matrices_cellular, FC_matrices_nodes = [], []
cell_coordinates = []

for i, folder in tqdm(enumerate(datasets)):
    
    data = load_data(folder)
    
    centroids = data['centroids_atlas']
    
    in_tectum = mask_tectum[centroids[:, 2], centroids[:, 1], centroids[:, 0]] > 0
    centroids_tectum = centroids[in_tectum]
    dff_tectum = data['timeseries'][in_tectum]
    dff_tectum = compute_dff_using_minfilter(dff_tectum, window=120, sigma1=3, sigma2=60)
    dff_tectum = filter_timeseries(dff_tectum, 2)

    #dff_tectum = spatial_smoothing(dff_tectum, centroids_tectum, sigma=20) # Used for tests

    corrs = np.corrcoef(dff_tectum)
    corrs[np.diag_indices(corrs.shape[0])] = 0    
    FC_matrices_cellular.append(corrs)
    cell_coordinates.append(centroids_tectum)

    nn, _ = find_nearest_neighbors(centroids_tectum, nodes_tectum)
    node_timeseries = np.zeros((nodes_tectum.shape[0], dff_tectum.shape[1]))
    for i in range(nodes_tectum.shape[0]):
        neurons_in_node = (nn == i) # Neurons whose nearest neighbor corresponds to the tectum node i
        if np.any(neurons_in_node):
            if np.sum(neurons_in_node) == 1:
                node_timeseries[i] = dff_tectum[neurons_in_node]
            else:
                node_timeseries[i] = np.mean(dff_tectum[neurons_in_node], axis=0)   
                
    FC = np.corrcoef(node_timeseries)
    FC_matrices_nodes.append(FC)

#### Plotting correlation matrices, single-cell resolution

In [ ]:
fig, axes = plt.subplots(2, 6, figsize=(15, 5))

for i, ax in enumerate(axes.flatten()):
    order = np.argsort(cell_coordinates[i][:, 1])
    C = np.copy(FC_matrices_cellular[i])[order, :][:, order]
    ax.imshow(C, cmap='coolwarm', vmin=-0.1, vmax=0.1)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(f'Fish {i+1}')

#### Plotting correlation matrices, coarse-grained resolution

In [ ]:
order = np.argsort(nodes_tectum[:, 1])

fig, axes = plt.subplots(2, 6, figsize=(15, 5))
for i, ax in enumerate(axes.flatten()):
    C = np.copy(FC_matrices_nodes[i])[order, :][:, order]
    C[np.isnan(C)] = 0
    ax.imshow(C, cmap='coolwarm', vmin=-0.25, vmax=0.25)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(f'Fish {i+1}')

#### Single-fish geometric gradients, coarse-grained

In [ ]:
similarity_matrices_nodes = []

for i in tqdm(range(len(FC_matrices_nodes))):
    FC = np.copy(FC_matrices_nodes[i])
    excluded = np.where(np.isnan(FC_matrices_nodes[i][0]))[0]
    FC = np.delete(np.delete(FC, excluded, axis=0), excluded, axis=1)
    #FC[FC < 0] = 0
    
    gradients, _ = diffusion_mapping(np.abs(normalize(FC)), n_components=100)
    
    vertices = np.load('../Files/tectum_vertices_right.npy') * 40
    vertices = np.stack([vertices[:, 2], vertices[:, 1], vertices[:, 0]], axis=1)
    eigenmodes = np.load('../Files/tectum_eigenmodes_right.npy')[1:]
    
    d = compute_distances(vertices, np.delete(nodes_tectum, excluded, axis=0))
    eigenmodes = eigenmodes[:, np.argmin(d, axis=0)]
    mode_similarity, mapping = compute_mode_similarity_matrix(eigenmodes[:30], gradients.T[:30], return_mapping=True)

    similarity_matrices_nodes.append(mode_similarity)

In [ ]:
order = np.argsort(nodes_tectum[:, 1])

fig, axes = plt.subplots(2, 6, figsize=(15, 5))
for i, ax in enumerate(axes.flatten()):
    ax.imshow(similarity_matrices_nodes[i], cmap='Reds', vmin=0, vmax=1)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(f'Fish {i+1}')

# Rendering supplementary figure

In [ ]:
fig = PaperFigure(figsize=(7, 7))

fig.set_tick_length(1)
fig.set_font_size(6)
fig.add_background()

# Adding axes ----------------------------------------------------------------------------

w = 1.2
pad = (7 - 5 * w) / 4

for i in range(5):
    fig.add_axes(f'similarity_matrix{i}', (i * (w + pad), 0), w, w)

w = 1
pad = (7 - 6 * w) / 5
for i in range(6):
    fig.add_axes(f'correlations{i}', (i * (w + pad),  1.75), w, w)
    fig.add_axes(f'correlations{i+6}', (i * (w + pad),  3), w, w)

for i in range(6):
    fig.add_axes(f'similarity_fish{i}', (i * (w + pad),  4.5), w, w)
    fig.add_axes(f'similarity_fish{i+6}', (i * (w + pad),  5.75), w, w)

fig.set_line_thickness(0.5)

# Filling axes ---------------------------------------------------------------------------

for i in range(5):
    ax = fig.axes[f'similarity_matrix{i}']
    ax.imshow(np.abs(matrices[i]), vmin=0.1, vmax=1, cmap='Reds')
    ax.set_xticks([])
    ax.set_yticks([])

for i in range(12):
    ax = fig.axes[f'correlations{i}']
    C = np.copy(FC_matrices_nodes[i])[order, :][:, order]
    C[np.isnan(C)] = 0
    ax.imshow(C, cmap='coolwarm', vmin=-0.4, vmax=0.4)
    ax.set_xticks([])
    ax.set_yticks([])

for i in range(12):
    ax = fig.axes[f'similarity_fish{i}']
    ax.imshow(similarity_matrices_nodes[i], cmap='Reds', vmin=0, vmax=1)
    ax.set_xticks([])
    ax.set_yticks([])

fig.save('../Figures/supp_coarsegraining_incomplete.svg')

fig.show()